In [4]:
!pip install neo4j
!pip install networkx
!pip install win10toast
!pip install graphdatascience[networkx]
!pip install ipywidgets

     -------------------------------------- 203.0/203.0 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280768 sha256=f4d0aa926dccba8ed7b63dc5f3ec6d596cf747eeec72c736fb05462f0e401606
  Stored in directory: c:\users\patri\appdata\local\pip\cache\wheels\db\9e\35\52396d2abf25000bf8bc4e88c93a06a427d9d24c3c9927846a
Successfully built neo4j
   ---------------------------------------- 24.9/24.9 MB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 1.6/1.6 MB 5.8 MB/s eta 0:00:00


In [186]:
from neo4j import GraphDatabase
import pandas as pd
from pyvis.network import Network

In [188]:
# Preparamos la sesión para acceder a neo4j
driver = GraphDatabase.driver('neo4j://localhost:7687', auth=("neo4j", "BDII2023"))
session = driver.session()

### MAPAMUNDI

In [189]:
mapamundi_query = "MATCH p=() - [:SHORTEST_PATH] - () RETURN p"

In [190]:
def mapamundi(session, query):
    results = session.run(query)

    net = Network(notebook=True)

    unique_areas = {}
    node_id = 0

    # Configuramos los colores de los nodos y las aristas
    node_color = "#84A98C"  
    node_border_color = "#52796F"  
    node_highlight_border_color = "#2B7CE9"  
    node_highlight_color = "#CAD2C5"  

    edge_color = "#354F52"  

    for record in results:
        path = record['p']
        start = dict(path.start_node)["name"]
        end = dict(path.end_node)["name"]
        weight = dict(path.relationships[0])["weight"]

        if start not in unique_areas:
            node_id += 1
            unique_areas[start] = node_id
            net.add_node(node_id, label=start, color=node_color, border=node_border_color,
                        highlight_border=node_highlight_border_color, highlight_color=node_highlight_color)

        if end not in unique_areas:
            node_id += 1
            unique_areas[end] = node_id
            net.add_node(node_id, label=end, color=node_color, border=node_border_color,
                        highlight_border=node_highlight_border_color, highlight_color=node_highlight_color)
        
        net.add_edge(unique_areas[start], unique_areas[end], value=weight, title=str(weight), 
                    color=edge_color, smooth=True)

    net.show("mapamundi.html")

In [191]:
mapamundi(session=session, query=mapamundi_query)

mapamundi.html


### LISTADO DE MAZMORRAS

In [212]:
listado_mazmorras_query = "MATCH (d:Room) \
                           WHERE d.dungeon_name IS NOT NULL \
                           WITH DISTINCT d.dungeon_name AS dungeon_name, collect(d) AS rooms \
                           UNWIND rooms AS room \
                           MATCH (room)-[:IS_CONNECTED]-(a:Area) \
                           RETURN dungeon_name, collect(DISTINCT a.name) AS connected_areas"
conexions = "MATCH p=(r:Room) - [c:IS_CONNECTED] - (a:Area) \
            RETURN c"

In [216]:
def listado_mazmorras(session, query):
    results = session.run(query).data()

    net = Network(notebook=True, height="800px", width="100%")

    unique_areas = {}
    dungeons = {}
    node_id = 0

    # Agregar nodos al gráfico
    for result in results:
        node_id += 1
        dungeon = result['dungeon_name']
        dungeons[dungeon] = node_id
        connected_areas = result['connected_areas']

        net.add_node(node_id, label=dungeon, color='#A6BDDC', border='#88A7D0',
                        highlight_border='#7398C4', highlight_color='#688DB9')
        
        for area in connected_areas:
            if area not in unique_areas:
                node_id += 1
                unique_areas[area] = node_id

                net.add_node(node_id, label=area, color='#FFFEBA', border='#FEFDA2',
                        highlight_border='#F7F791', highlight_color='#EBEC86')
                
            net.add_edge(dungeons[dungeon], unique_areas[area], edge_color='#808080')

    net.show("listado_mazmorras.html")
        

In [218]:
listado_mazmorras(session=session, query=listado_mazmorras_query)

listado_mazmorras.html


### Minimapa de mazmorra

In [219]:
mazmorra = "MATCH (entrance:Room {dungeon_name: $dungeon_name})-[:IS_CONNECTED]->(:Area) \
            MATCH (exit:Room {dungeon_name: $dungeon_name}) <- [:IS_CONNECTED] - (:Area) \
            WHERE entrance <> exit \
            OPTIONAL MATCH  (entrance) - [:CONTAINS] - (m1:Monster) \
            OPTIONAL MATCH  (entrance) - [:CONTAINS] - (l1:Loot) \
            OPTIONAL MATCH  (exit) - [:CONTAINS] - (m2:Monster) \
            OPTIONAL MATCH  (exit) - [:CONTAINS] - (l2:Loot) \
            MATCH (r:Room {dungeon_name: $dungeon_name}) - [:IS_CONNECTED] - (:Room)\
            WHERE r <> entrance AND r <> exit \
            MATCH (r) - [:CONTAINS] - (m:Monster), \
                  (r) - [:CONTAINS] - (l:Loot) \
            RETURN entrance, exit, r, m, l, m1, l1, m2, l2"

conexions = "MATCH p=(r:Room {dungeon_name: $dungeon_name}) - [:IS_CONNECTED] - (r2:Room {dungeon_name: $dungeon_name}) \
              RETURN p"

dungeon_name = 'Benton, Necropolis of the Clumsy Presidents'

In [220]:
def minimapa_mazmorra(session, query, relationships, dungeon_name):
    results = session.run(query, dungeon_name=dungeon_name).data()
    conexions = session.run(relationships, dungeon_name=dungeon_name).data()

    net = Network(notebook=True, height="800px", width="100%")

    unique_areas = {}
    unique_loots = {}
    unique_monsters = {}
    node_id = 0

    for result in results:
        entrance = result['entrance']['room_name']
        exit = result['exit']['room_name']
        room = result['r']['room_name']

        if entrance not in unique_areas:
            node_id += 1
            unique_areas[entrance] = node_id

            net.add_node(node_id, label=str(f'{entrance} - entrance'), color='#618E3C', border='#B5CE88',
                        highlight_color='#8C924F', highlight_border='#AAAE75')

        if exit not in unique_areas:
            node_id += 1
            unique_areas[exit] = node_id

            net.add_node(node_id, label=str(f'{exit} - exit'), color='#B5CE88', border='#618E3C',
                        highlight_border='#8C924F', highlight_color='#3AAAE75')

        if room not in unique_areas:
            node_id += 1
            unique_areas[room] = node_id

            net.add_node(node_id, label=room, color='#FFCAD4', border='#F4ACB7',
                        highlight_border='#E49CA5', highlight_color='#EEADA8')

        # Monstruos de las salas, la entrada y la salida
        if result['m'] is not None:
            if  result['m']['name'] not in unique_monsters:
                node_id += 1
                m = result['m']['name']
                unique_monsters[m] = node_id

                nivel = result['m']['level']

                net.add_node(node_id, label=str(f'Nombre: {m} \n Nivel: {nivel}'), color='#A6BDDC', border='#88A7D0',
                        highlight_border='#7398C4', highlight_color='#688DB9')

            net.add_edge(unique_areas[room], node_id, edge_color='#5D83AE')

        if result['m1'] is not None:
            if result['m1']['name'] not in unique_monsters:
                node_id += 1
                m1 = result['m1']['name']
                unique_monsters[m1] = node_id

                nivel = result['m1']['level']

                net.add_node(node_id, label=str(f'Nombre: {m1} \n Nivel: {nivel}'), color='#A6BDDC', border='#88A7D0',
                        highlight_border='#7398C4', highlight_color='#688DB9')

            net.add_edge(unique_areas[entrance], node_id, edge_color='#5D83AE')

        if result['m2'] is not None:
            if result['m2']['name'] not in unique_monsters:
                node_id += 1
                m2 = result['m2']['name']
                unique_monsters[m2] = node_id

                nivel = result['m2']['level']

                net.add_node(node_id, label=str(f'Nombre: {m2} \n Nivel: {nivel}'), color='#A6BDDC', border='#88A7D0',
                        highlight_border='#7398C4', highlight_color='#688DB9')

            net.add_edge(unique_areas[exit], node_id, edge_color='#5D83AE')

        #Tesoros de las salas, la entrada y la salida
        if result['l'] is not None:
            if result['l']['name'] not in unique_loots:
                node_id += 1
                l = result['l']['name']
                unique_loots[l] = node_id

                gold = result['l']['gold']

                net.add_node(node_id, label=str(f'Nombre: {l} \n Nivel: {gold}'), color='#FFFEBA', border='#FEFDA2',
                        highlight_border='#F7F791', highlight_color='#EBEC86')

            net.add_edge(unique_areas[room], node_id, edge_color='#DFE07B')

        if result['l1'] is not None:
            if result['l1']['name'] not in unique_loots:
                node_id += 1
                l1 = result['l1']['name']
                unique_loots[l1] = node_id

                gold = result['l1']['gold']

                net.add_node(node_id, label=str(f'Nombre: {l1} \n Nivel: {gold}'), color='#FFFEBA', border='#FEFDA2',
                        highlight_border='#F7F791', highlight_color='#EBEC86')

            net.add_edge(unique_areas[entrance], node_id, edge_color='#DFE07B')

        if result['l2'] is not None:
            if result['l2']['name'] not in unique_loots:
                node_id += 1
                l2 = result['l2']['name']
                unique_loots[l2] = node_id

                gold = result['l2']['gold']


                net.add_node(node_id, label=str(f'Nombre: {l2} \n Nivel: {gold}'), color='#FFFEBA', border='#FEFDA2',
                        highlight_border='#F7F791', highlight_color='#EBEC86')
            
            net.add_edge(unique_areas[exit], node_id, edge_color='#DFE07B')

    for relation in conexions:
        # Añadimos las conexiones entre habitaciones
        from_room = relation['p'][0]['room_name']
        if from_room not in unique_areas:
            node_id += 1
            unique_areas[from_room] = node_id

            net.add_node(node_id, label=from_room,  color='#FFCAD4', border='#F4ACB7',
                        highlight_border='#E49CA5', highlight_color='#EEADA8')
        
        to_room = relation['p'][2]['room_name']
        if to_room not in unique_areas:
            node_id += 1
            unique_areas[to_room] = node_id

            net.add_node(node_id, label=to_room,  color='#FFCAD4', border='#F4ACB7',
                        highlight_border='#E49CA5', highlight_color='#EEADA8')
            
        net.add_edge(unique_areas[from_room], unique_areas[to_room])

    net.show("mazmorra.html")
    

In [221]:
minimapa_mazmorra(session=session, query=mazmorra, relationships=conexions, dungeon_name=dungeon_name)

mazmorra.html
